## LSTM-CNN Filter Experiment

In [ ]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### RNN Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(4,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(embedded_sequences)
l_cov1= Conv1D(24, 9, activation='relu')(l_lstm1)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_drop1 = Dropout(0.3)(l_pool1)
l_flat = Flatten()(l_drop1)
l_dense = Dense(16, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

In [ ]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [ ]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [ ]:
model.summary()
print("Training Progress:")
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                    epochs=50, batch_size=64,
                    callbacks=[lr_schedule])

In [ ]:
import pandas as pd
pd.DataFrame(history.history).to_csv("history-9.csv")
#model.save('ltsm-c-3.h5')